# 05. 블러와 이진화

In [ ]:
import cv2 as cv
import numpy as np
DOG_PATH = "../images/dog.jpg"
BOOK_PATH = "../images/book.jpg"

## 5-1. 블러
- 이미지를 흐리게 하는 효과
- ```cv2.blur(img, kernel)``` : 단순 평균 방식
    - kernel : 블러값을 계산하기 위한 영역
- ```cv2.GaussianBlur(img, kernel, 표준편차)``` : 가우시안 평균 방식
    - 표준편차에 0 넣으면 자동 계산
- ```cv2.medianBlur(img, kernel)``` : 중앙값 사용 방식

In [ ]:
img = cv.imread(DOG_PATH)

# 평균 블러
blur_avg = cv.blur(img, (5,5))

# 가우시안 블러
blur_gaussian = cv.GaussianBlur(img, (5,5), 0)

cv.imshow("Blur", blur_avg)
cv.imshow("Gaussian", blur_gaussian)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
# 커널 사이즈 변화에 따른 차이

img = cv.imread(DOG_PATH)

kernel_3 = cv.GaussianBlur(img, (3,3), 0)
kernel_5 = cv.GaussianBlur(img, (5,5), 0)
kernel_7 = cv.GaussianBlur(img, (7,7), 0)

cv.imshow("kernel_3", kernel_3)
cv.imshow("kernel_5", kernel_5)
cv.imshow("kernel_7", kernel_7)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
# 표준편차 변화에 따른 차이

img = cv.imread(DOG_PATH)

sigma_1 = cv.GaussianBlur(img, (0,0), 1)
sigma_2 = cv.GaussianBlur(img, (0,0), 2)
sigma_3 = cv.GaussianBlur(img, (0,0), 3)

cv.imshow("sigma_1", sigma_1)
cv.imshow("sigma_2", sigma_2)
cv.imshow("sigma_3", sigma_3)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

## 5-2. 이진화(Binarization)
- 특정 값을 기준으로 픽셀의 값을 0(검은색)또는 255(흰색)로 분류하는 것
- 분석을 단순화하기 위해 사용
- 특정 관심 영역을 분리하는데 용이함

### Threshold : 임계값, 문턱값
- 기준값의 역할을 함
- 빛의 밝기를 기준으로 픽셀 값을 변환
- ```cv2.threshold(img, threshold, maxValue, type)```
    - return / binary 값 반환
    - return = threshold 값
    - binary = 이미지

In [ ]:
img = cv.imread(BOOK_PATH, cv.IMREAD_GRAYSCALE)

ret, binary = cv.threshold(img, 127, 255, cv.THRESH_BINARY)

cv.imshow("img", img)
cv.imshow("binary", binary)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

### 트랙바(Track bar)
- ```cv2.createTrackbar(trackbarName, name, startValue, maxValue, callback)``` : 트랙바 생성
- ```cv2.getTrackbarPos(trackbarName, name)``` : 트랙바의 값을 가져옴

In [11]:
img = cv.imread(BOOK_PATH, cv.IMREAD_GRAYSCALE)

# 트랙바 생성 이전에 창이 만들어져 있어야함
name = "Threshold"
cv.namedWindow(name)

# 트랙바 생성
trackbar_name = "threshold"
cv.createTrackbar(trackbar_name, name, 127, 255, lambda x:x)

while True:
    threshold = cv.getTrackbarPos(trackbar_name, name)
    ret, binary = cv.threshold(img, threshold, 255, cv.THRESH_BINARY)

    cv.imshow(name, binary)

    if cv.waitKey(1) == ord("q"):
        break

cv.destroyAllWindows()
cv.waitKey(1)

-1

In [ ]:
img = cv.imread(DOG_PATH)

name = "Resize"
cv.namedWindow(name)

trackbar_name = "scale(%)"
cv.createTrackbar(trackbar_name, name, 100, 200, lambda x:x)

while True:
    scale = cv.getTrackbarPos(trackbar_name, name)
    
    if scale == 0:
        scale = 1

    width = int(img.shape[1] * scale / 100)
    height = int(img.shape[0] * scale / 100)
    resized = cv.resize(img, (width, height))

    cv.imshow(name, resized)

    if cv.waitKey(1) == ord("q"):
        break

cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
# 실습2-2. 컬러 팔레트 만들기

img = np.zeros((460, 640, 3), dtype=np.uint8)

name = "COLOR"
cv.namedWindow(name)

cv.createTrackbar("B", name, 0, 255, lambda x:x)
cv.createTrackbar("G", name, 0, 255, lambda x:x)
cv.createTrackbar("R", name, 0, 255, lambda x:x)

trackbar_name = "0:OFF\n1:ON"
cv.createTrackbar(trackbar_name, name, 1, 1, lambda x:x)

while True:
    B = cv.getTrackbarPos("B", name)
    G = cv.getTrackbarPos("G", name)
    R = cv.getTrackbarPos("R", name)

    switch = cv.getTrackbarPos(trackbar_name, name)
    if switch == 1:
        img[:] = (B,G,R)

    cv.imshow(name, img)

    if cv.waitKey(1) == ord("q"):
        break

cv.destroyAllWindows()
cv.waitKey(1)

## 5-3. 적응형 이진화
- 영역별로 서로 다른 임계값을 적용하는 이진화
- ```cv2.adaptiveThreshold(img, maxValue, adaptiveMethod, thresholdType, blockSize, C)```
    - adaptiveMethod : 적응형 임계값 알고리즘
    - blockSize : 픽셀의 임계값을 계산하는 데 사용되는 픽셀 근처의 크기(1보다 큰 홀수)
    - C : 평균 또는 가중 평균에서 뺀 상수 (임계값 미세 조정)

In [14]:
img = cv.imread(BOOK_PATH, cv.IMREAD_GRAYSCALE)

name = "Adaptive Threshold"
cv.namedWindow(name)

cv.createTrackbar("block_size", name, 25, 100, lambda x:x) 
cv.createTrackbar("C", name, 1, 10, lambda x:x)

while True:
    block_size = cv.getTrackbarPos("block_size", name)
    C = cv.getTrackbarPos("C", name)

    if block_size <= 1:
        block_size = 3
    
    if block_size % 2 == 0:
        block_size += 1

    binary = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, block_size, C)

    cv.imshow(name, binary)

    if cv.waitKey(1) == ord("q"):
        break

cv.destroyAllWindows()
cv.waitKey(1)

-1

## 5-4. 오츠 알고리즘
- 최적의 threshold를 찾는 알고리즘
- 각 그룹 내부의 분산(값의 퍼짐)을 최소화하는 임계값을 찾는 것을 목표로 함

In [17]:
img = cv.imread(DOG_PATH, cv.IMREAD_GRAYSCALE)

ret_1, binary = cv.threshold(img, 127, 255, cv.THRESH_BINARY)
ret_2, otsu = cv.threshold(img, -1, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)

print(ret_1, ret_2)

cv.imshow("img", binary)
cv.imshow("otsu", otsu)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

127.0 120.0


-1